In [1]:
# imports
#import tkinter as tk
#from tkinter import simpledialog
import requests 
from bs4 import BeautifulSoup
from icalendar import Calendar, Event, vCalAddress, vText
import datetime
from pathlib import Path
import pandas as pd
import os
from IPython.display import display, HTML

os.environ['TZ'] = 'Europe/Berlin'

#requesting url and name from user
'''
ROOT = tk.Tk()
ROOT.withdraw()
urlInput = simpledialog.askstring(title="url",
                                  prompt="url der DFN-Umfrage:")
ROOT.withdraw()
nameInput = simpledialog.askstring(title="name",
                                  prompt="Name der Person (genau wie in der DFN-Tabelle):")
'''                                

urlInput = input("url der DFN-Umfrage")
nameInput = input("Name der Person (genau wie in der DFN-Tabelle):")

# downloading and processing data
url = urlInput
url = url.replace(".de/en/p/", ".de/de/p/")
website = requests.get(url)
results = BeautifulSoup(website.content, 'html.parser')
dates_table = pd.read_html(results.prettify())[0]
dates_table.set_index('Unnamed: 0', inplace=True)

# creating calender and populating whith events
summary = results.find(id="page-title").text
name = nameInput

months = {"Januar": 1, "Februar": 2, "März": 3, "April": 4, "Mai": 5, "Juni": 6, "Juli": 7, "August": 8, "September": 9, "Oktober": 10, "November": 11, "Dezember": 12}

c = Calendar()
c.add('prodid', '-//My calendar product//example.com//')
c.add('version', '2.0')
for columnname in dates_table:
    column = dates_table[columnname]
    if (column.get(name)=='Ja'):
        year = column.name.split()[1][0:4]
        month = months[column.name.split()[0]]
        day = column.get(column.index[0]).split()[1]
        start = column.get(column.index[1]).split('->')[0]
        end = column.get(column.index[1]).split('->')[1]
        d_start = str(month)+ '/' + str(day) +'/' + str(year) + ' ' + start
        start_date = datetime.datetime.strptime(
          d_start, '%m/%d/%Y %H:%M')
        d_end = str(month)+ '/' + str(day) +'/' + str(year) + ' ' + end
        end_date = datetime.datetime.strptime(
          d_end, '%m/%d/%Y %H:%M')
        event = Event()
        event.add('dtstart', start_date)
        event.add('dtend', end_date)
        event.add('summary', summary)
        event.add('url', url)
        event.add('status', 'TENTATIVE')
        event.add('description', summary)
        c.add_component(event)

#export ics
f = open(os.path.join('dates.ics'), 'wb')
f.write(c.to_ical())
f.close()

#show download link
HTML(f'<a href="dates.ics" download>ics runterladen</a>')

url der DFN-Umfrage https://terminplaner6.dfn.de/p/dd909abc6f030287013be02e4767199b-370621
Name der Person (genau wie in der DFN-Tabelle): Ben
